# [모듈 2.1] 모델 훈련 스텝 및 모델 등록 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 환경 세팅
- 1. 모델 훈련 스텝 개발 및 실행
- 2. 모델 평가 파일 다운로드
- 3. SageMaker Pipeline에서 실행
- 4. 모델 레지스트리에서 모델 등록 확인
- 5. 리소스 정리: 파이프라인

---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---

### 참고
- [Amazon SageMaker Model Building Pipeline](https://sagemaker.readthedocs.io/en/v2.92.2/amazon_sagemaker_model_building_pipeline.html#amazon-sagemaker-model-building-pipeline)
- [아마존 SageMaker 모델 구축 파이프라인](https://us-east-1.console.aws.amazon.com/codesuite/codepipeline/pipelines/ncf-training-code-pipeline/view?region=us-east-1)
- 모델 등록 스텝은 여기를 참조 바랍니다. --> [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)


# 0. 환경 세팅


In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm_client = boto3.client("sagemaker")



저장된 변수를 확인 합니다.

In [2]:
%store -r s3_input_data_uri
%store -r bucket
%store -r project_prefix



데이타가 존재하는지 확인 합니다.

In [3]:
! aws s3 ls {s3_input_data_uri} --recursive

Unable to locate credentials. You can configure credentials by running "aws configure".


# 1. 모델 훈련 스텝 개발 및 실행
---



## 1.1. 훈련 및 테스트 데이터를 S3 로 지정

In [4]:
s3_inputs = {
            'train': f'{s3_input_data_uri}',
            'test': f'{s3_input_data_uri}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


In [5]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}        
    ]


In [6]:
# from datetime import datetime
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

hyperparameters = {'epochs': 1, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  


host_estimator = PyTorch(
    entry_point="train_metric.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=hyperparameters,
    metric_definitions = metric_definitions

)
host_estimator.fit(s3_inputs, 
                   # experiment_config = experiment_config, # 실험 설정 제공                   
                   wait=False)    



In [7]:
## 마지막 estimator의 로그 출력
host_estimator.logs()

2022-07-08 10:29:59 Starting - Starting the training job...
2022-07-08 10:30:28 Starting - Preparing the instances for trainingProfilerReport-1657276199: InProgress
.........
2022-07-08 10:31:59 Downloading - Downloading input data
2022-07-08 10:31:59 Training - Downloading the training image.................................
2022-07-08 10:37:14 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-08 10:37:14,164 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-08 10:37:14,188 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-08 10:37:14,195 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-08 10:37:14,665 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip instal

# 2. 모델 평가 파일 다운로드

이 단계는 model registry 에 모델을 등록시에 모델의 성능 메트릭을 넣기위한 단계 입니다. 이후 model restry 단계에서 치종 결과인 메트릭의 S3 경로를 사용할 예정 입니다.

- 아래 작업은 모델 훈련을 하면서 모델의 평가 (Train:Auc, Validation:Auc) 의 값을 'metrics.json' 에 저장을 하고, 도커 안의 환경 변수가 가르키는 곳에  (SM_OUTPUT_DATA_DIR=/opt/ml/output/data) 저장을 합니다.
- metrics.json 파일을 훈련이 끝난 후에 아래와 같은 경로 (에: s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/sagemaker-xgboost-2022-06-23-12-25-31-125/output/output.tar.gz) 에 업로딩 합니다. 
- 이 파일을 로컬 폴더 (에: ./output) 에 다운로드 하고, 압축을 해제 합니다.
- 'metrics.json' 파일을 S3 에 업로드 하여 s3_uri 경로를 저장 합니다.
- s3_uri 는 model registry 에 모델을 등록시에 인자로 제공하여 , 모델의 성능 메트릭을 관리 합니다.

In [8]:
output_artifact_path = f"{host_estimator.output_path}{host_estimator.latest_training_job.job_name}/output/output.tar.gz"
print(f"output_artifact_path: {output_artifact_path}")

output_artifact_path: s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-07-08-10-29-59-553/output/output.tar.gz


In [9]:
! aws s3 ls {output_artifact_path}

2022-07-08 10:41:26        205 output.tar.gz


In [10]:
output_data_dir = './output'
os.makedirs(output_data_dir, exist_ok=True)

In [11]:
%%sh -s {output_artifact_path} {output_data_dir}

output_artifact_path=$1
output_data_dir=$2

echo $output_artifact_path
echo $output_data_dir


# 모델을 S3에서 로컬로 다운로드
aws s3 cp $output_artifact_path $output_data_dir

# # 모델 다운로드 폴더로 이동
cd $output_data_dir

# # 압축 해제
tar -xvf output.tar.gz  

rm -rf output.tar.gz  

s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-07-08-10-29-59-553/output/output.tar.gz
./output
download: s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-07-08-10-29-59-553/output/output.tar.gz to output/output.tar.gz
metrics.json


로컬의 metrics.json 을 S3에 업로딩 합니다.

In [12]:
metric_path = os.path.join(output_data_dir, 'metrics.json')
metric_data_uri = f"s3://{bucket}/{project_prefix}"

metric_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=metric_path, 
    desired_s3_uri=metric_data_uri,    
)
print(metric_data_uri)



s3://sagemaker-us-east-1-057716757052/NCFModel/metrics.json


# 3. SageMaker Pipeline에서  실행 
- 모델 훈련 스텝과 모델 등록 스텝 두가지를 실행합니다.
    - 두 개의 단계가 서로 의존성이 있기에, 두 개의 단계를 연결을 합니다.

---



### 모델 빌딩 파이프라인 변수 생성



In [13]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

s3_data_loc = ParameterString(
    name="InputData",
    default_value=s3_input_data_uri,
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)



### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [14]:
host_hyperparameters = {'epochs': 1, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

from sagemaker.pytorch import PyTorch

estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
print("estimator_output_path: \n", estimator_output_path)


instance_type = 'ml.p3.2xlarge'
instance_count = 1

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    output_path = estimator_output_path,    
    framework_version='1.8.1',
    py_version='py3',
    disable_profiler = True,
    instance_count=instance_count,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions
    
)



estimator_output_path: 
 s3://sagemaker-us-east-1-057716757052/NCFModel/training_jobs


이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.

In [15]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name= "NCF-Training",
    estimator=host_estimator,
    inputs={
        "train": TrainingInput(
            s3_data= s3_data_loc
        ),
        "test": TrainingInput(
            s3_data= s3_data_loc
        ),        
    }
)

### 모델 등록 스텝
- 모델의 성능 지표 (metrics.json) 를 등록하기 위해 ModelMetrics 오브젝트 생성.
    - [Python SDK, class sagemaker.model_metrics.ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.MetricsSource)
- 모델들이 저장이 될 그룹 이름을 제공하고, 모델 등록 스텝을 정의 합니다.
- 모델 등록 단계의 개발자 가이드 
    - [모델 등록기 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model)
    - [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)
- 모델 그룹 리스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)   

In [16]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

# 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 에 모데 버전 등록시에 삽입함
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri= metric_data_uri,
        content_type="application/json"
    )
)



In [17]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

from sagemaker.workflow.step_collections import RegisterModel

step_register = RegisterModel(
    name= "NCF-Model-Registry",
    estimator=host_estimator,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.p2.xlarge", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)



### 모델 빌딩 파이프라인 정의
- 파이프라인과 실험(Experiment)가 통합이 되었습니다. 이를 위한 실험 설정 파일을 같이 제공합니다.

In [18]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        # processing_instance_type, 
        # processing_instance_count,
        # training_instance_type,        
        # training_instance_count,                
        s3_data_loc,
        # model_eval_threshold,
        model_approval_status,        
    ],
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_train, step_register],
)




In [19]:
import json

definition = json.loads(pipeline.definition())
# definition

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [20]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncfmodel',
 'ResponseMetadata': {'RequestId': '0a7baae2-4c50-4075-a134-80c3ce1b5799',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a7baae2-4c50-4075-a134-80c3ce1b5799',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '76',
   'date': 'Fri, 08 Jul 2022 10:41:51 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [21]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [22]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncfmodel',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncfmodel/execution/wby53uv0bh27',
 'PipelineExecutionDisplayName': 'execution-1657276912896',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 7, 8, 10, 41, 52, 801000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 7, 8, 10, 41, 52, 801000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '0ba74a8d-868b-4c91-bb7e-7f4836ef05ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0ba74a8d-868b-4c91-bb7e-7f4836ef05ae',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '381',
   'date': 'Fri, 08 Jul 2022 10:41:52 GMT'},
  'RetryAttempts': 0}}

In [23]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [24]:
execution.list_steps()

[{'StepName': 'NCF-Model-Registry',
  'StartTime': datetime.datetime(2022, 7, 8, 10, 53, 24, 251000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 8, 10, 53, 25, 342000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/14'}}},
 {'StepName': 'NCF-Training',
  'StartTime': datetime.datetime(2022, 7, 8, 10, 41, 53, 944000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 8, 10, 53, 18, 765000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:training-job/pipelines-wby53uv0bh27-ncf-training-mrmjbt7hlh'}}}]

# 4. 모델 레지스트리에서 모델 등록 확인
위에서 등록한 모델 그룹 이름을 통해서 어떤 모델이 등록되었는지를 확인 합니다.
스튜디오에서 실제 등록된 성능 지표를 확인 할 수 있습니다.


- 등록된 모델 버전에 대한 보기 --> [모델 버전의 세부 정보 보기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry-details.html)

In [25]:
# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'NCFModel',
   'ModelPackageVersion': 14,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/14',
   'CreationTime': datetime.datetime(2022, 7, 8, 10, 53, 25, 228000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'NCFModel',
   'ModelPackageVersion': 13,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/13',
   'CreationTime': datetime.datetime(2022, 7, 8, 10, 48, 45, 712000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'NCFModel',
   'ModelPackageVersion': 12,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/12',
   'CreationTime': datetime.datetime(2022, 7, 7, 12, 53, 11, 668000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'M

#### 등록된 모델 버전의 상세 정보 확인

In [26]:
ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

{'ModelPackageGroupName': 'NCFModel',
 'ModelPackageVersion': 14,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/14',
 'CreationTime': datetime.datetime(2022, 7, 8, 10, 53, 25, 228000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8.1-gpu-py3',
    'ImageDigest': 'sha256:392884aa51bb8578a98ce5d3648a0a9f36c0389113480b0e869333909d986d6a',
    'ModelDataUrl': 's3://sagemaker-us-east-1-057716757052/NCFModel/training_jobs/pipelines-wby53uv0bh27-NCF-Training-MrmJbt7Hlh/output/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m5.xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.p2.xlarge', 'ml.m5.xlarge'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': False,
 'M

# 5. 리소스 정리: 파이프라인
- 위에서 생성한 파이프라인을 제거 합니다.
- isDeletePipeline=False, verbose=Fasle
    - 파이프라인을 지우지 않고, 존재하는지 확인 합니다.
- isDeletePipeline=False, verbose=True
    - 파이프라인의 정의를 자세하 확인 합니다.
- isDeletePipeline=True, verbose=True or False
    - 파이프라인을 삭제 합니다.

In [27]:
# from src.pipeline_util import clean_pipeline

# # clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
# clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=True, verbose=False)   